<a href="https://colab.research.google.com/github/DenizY98/pydemo/blob/main/Fallstudie1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prozessoptimierung - Fallstudie

##PIP, OR-Tools und Pandas

In [1]:
# Importieren der benötigten Bibliotheken
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp
import pandas as pd
# Initialisieren des Solvers
solver = pywraplp.Solver.CreateSolver('SCIP')

In [2]:
! git clone https://github.com/AlexKressner/Industrielles_Management

fatal: destination path 'Industrielles_Management' already exists and is not an empty directory.


In [3]:
path = "Industrielles_Management/Daten/Fallstudie/"

In [4]:
#dataframes laden aus Daten der CSV-Dateien
nachfrage_df = pd.read_csv(f"{path}/FLINK_Nachfrage.csv", sep=";", index_col=0) # Da Indizes in Datenquelle so festgelegt
standorte_df = pd.read_csv(f"{path}/FLINK_Standorte.csv", sep=";")

##Ziel:
Maximierung der bedienten Nachfrage innerhalb der vorgegebenen Lieferzeit und des Budgets.


##Daten:


###Indexmengen:
$ Q = \{q \mid q = (i,j), i \in \{0, \ldots, 12\}, j \in \{0, \ldots, 12\}\} $
Menge aller Quadranten $ q $ als $( i , j )$ Tupel


$ S = \{s \mid s \in \{0, \ldots, 9\}\} $
Menge aller potentieller DarkStores $ s $

In [5]:
Q = [(i, j) for i in range(nachfrage_df.shape[0]) for j in range(nachfrage_df.shape[1])]  # Menge aller Quadranten als (i, j) Paare
S = standorte_df['Potenzielle_Standorte'].to_list()  # Menge aller potentieller DarkStores

###Variablen:
$ x_s \in \{0,1\} \quad \forall s \in S $: Binärvariable, die angibt, ob am Standort $ s $ ein Dark Store eingerichtet wird $1$ oder nicht $0$.

$ y_{qs} \in \{0,1\} \quad \forall q \in Q, \forall s \in S $: Kontinuierliche Variable, die den %ualen Anteil der Nachfrage eines Quadranten $ q (i,j) $ repräsentiert, der von  Darkstore $ s $ bedient wird.


In [6]:
x = {}
for s in S:
  x[s] = solver.BoolVar(f'x[{s}]')

In [7]:
y = {}
for s in S:
    for q in Q:
        y[q, s] = solver.NumVar(0, 1, f'y[{q},{s}]')


###Dictionaries:
- Lagerumschlagsleistung je potentiellem Standort $ s $
- Errichtungskosten je potentiellem Standort $ s $
- Koordinaten je potentiellem Standort $ s $

In [8]:
#Standorte Dataframe
Lagerumschlagleistung = standorte_df.set_index('Potenzielle_Standorte')['Lagerumschlagleistung'].to_dict()
Errichtungskosten = standorte_df.set_index('Potenzielle_Standorte')['Errichtungskosten'].to_dict()
Koordinaten = standorte_df.set_index('Potenzielle_Standorte')[['i_Koordinate','j_Koordinate']].to_dict('index')

###Parameter:

$ D_q $: Nachfrage im Quadranten $ q $ als Bestellungen pro Tag.

$ L_s $: Lagerumschlagleistung des Standorts $ s $ als Bestellungen pro Tag.

$ C_s $: Kosten für die Einrichtung eines Dark Stores am Standort $ s $.

$ B $: Gesamtbudget für die Einrichtung von Dark Stores.

$ T $: Maximale Lieferzeit $10 Minuten$.

$ A $: Maximale Anzahl von Quadranten, die ein Fahrradkurier bedienen kann $5 Quadranten$.


In [9]:
D = {q: nachfrage_df.iat[q[0],q[1]] for q in Q} # Nachfrage im Quadranten q als Bestellungen pro Tag
L = {s: Lagerumschlagleistung[s] for s in S}  # Lagerumschlagleistung des Standorts s als Bestellungen pro Tag
C = {s: Errichtungskosten[s] for s in S}  # Kosten für die Einrichtung eines Dark Stores am Standort s
B = 1000000  # Gesamtbudget für die Einrichtung von Dark Stores
G = 25  # Geschwindigkeit in KM/H
T = 10  # Maximale Lieferzeit 10 Minuten
TF1= 3  # Zeitfresser in Minuten Warenbereitstellung AE
TF2= 1  # Zeitfresser in Minuten Warenübergabe beim Kunden
TV= (T-TF1-TF2) # Verbleibende Zeit abzüglich Zeitfressern
A = round((TV/2)/0.25*G/60, None) # max Reichweite in Quadranten für eine Richtung

###Restriktionen:
####Nachfragerestriktionen: $
\sum_{s \in S} y_{qs} \leq 2 \quad \forall q \in Q $
Jeder Quadrant wird von höchstens zwei Dark Stores bedient
####Budgetrestriktion: $
\sum_{s \in S} x_s \cdot C_s \leq B $
Ein Dark Store kann nur eingerichtet werden, wenn das Budget dies zulässt
####Lagerumschlagleistungsrestriktionen: $
\sum_{q \in Q} y_{qs} \cdot D_{q} \leq x_s \cdot L_s \quad \forall s \in S $
Die Nachfrage eines Quadranten kann nur bedient werden, wenn die Lagerumschlagleistung ausreicht und ein Dark Store vorhanden ist
####Reichweitenrestriktion für Fahrradkuriere: $
\text{Strecke}(s, q) > A \Rightarrow y_{qs} = 0 \quad \forall q \in Q, \forall s \in S $
Die Anzahl der von einem Fahrradkurier zurückgelegten Quadranten $Strecke$ darf nie größer $A$ sein, falls bestätigt, dass Quadrant $q$ von Standort $s$ beliefert wird.

In [10]:
# Nachfragerestriktionen
for q in Q:
    solver.Add(solver.Sum([y[q, s] for s in S]) <= 2)
# Budgetrestriktion
solver.Add(solver.Sum([x[s] * C[s] for s in S]) <= B)

# Restriktionen für z

# Anpassung der Zuordnungsrestriktionen
for s in S:
    for q in Q:
        solver.Add(y[q, s] <= x[s])  # Ein Darkstore kann nur einen Quadranten bedienen, wenn er errichtet wurde
# Lagerumschlagleistungsrestriktionen
for s in S:
    solver.Add(solver.Sum([y[q, s] * D[q] for q in Q]) <= x[s] * L[s])
# # Lagerumschlagleistungsrestriktionen
# Reichweitenrestriktion für Fahrradkuriere
for s in S:
    for q in Q:
        # Berechnung der Manhattan-Distanz zwischen dem Standort s und dem Quadranten q
        distance = abs(Koordinaten[s]['i_Koordinate'] - q[0]) + abs(Koordinaten[s]['j_Koordinate'] - q[1])
        if distance > 5:
            solver.Add(y[q, s] == 0)

##Lösung:
###Zielfunktion:
Maximiere die gesamte bediente Nachfrage:

$ \text{Max} \sum_{sq} y_{qs} \cdot D_q $

In [11]:
# Ändern Sie die Zielfunktion
solver.Maximize(solver.Sum([y[q, s] * D[q] for q in Q for s in S]))
# Lösen des Modells
status = solver.Solve()

In [13]:
# Ausgabe der Lösung
if status == pywraplp.Solver.OPTIMAL:
    print('Lösung gefunden:')
    for s in S:
        if x[s].solution_value() == 1:
            print(f'Standort {s} wird eingerichtet.')
            for q in Q:
                if y[q, s].solution_value() == 1:
                    print(f'Quadrant {q} wird von Standort {s} bedient.')
else:
    print('Keine optimale Lösung gefunden.')

Lösung gefunden:
Standort 1 wird eingerichtet.
Quadrant (4, 10) wird von Standort 1 bedient.
Quadrant (6, 10) wird von Standort 1 bedient.
Quadrant (7, 7) wird von Standort 1 bedient.
Quadrant (7, 9) wird von Standort 1 bedient.
Quadrant (7, 12) wird von Standort 1 bedient.
Quadrant (8, 8) wird von Standort 1 bedient.
Quadrant (9, 8) wird von Standort 1 bedient.
Quadrant (10, 12) wird von Standort 1 bedient.
Quadrant (11, 12) wird von Standort 1 bedient.
Quadrant (12, 10) wird von Standort 1 bedient.
Standort 2 wird eingerichtet.
Quadrant (6, 4) wird von Standort 2 bedient.
Quadrant (7, 6) wird von Standort 2 bedient.
Quadrant (8, 6) wird von Standort 2 bedient.
Quadrant (9, 5) wird von Standort 2 bedient.
Quadrant (9, 8) wird von Standort 2 bedient.
Quadrant (10, 1) wird von Standort 2 bedient.
Quadrant (11, 3) wird von Standort 2 bedient.
Standort 3 wird eingerichtet.
Quadrant (6, 11) wird von Standort 3 bedient.
Quadrant (7, 8) wird von Standort 3 bedient.
Quadrant (7, 10) wird von 

### Darstellung als Matrix, DF oder MatPlot

In [26]:
import numpy as np
# Erstellen Sie eine leere Matrix der gleichen Größe wie Ihr Quadrantengitter
ergebnis_matrix = np.empty((14, 14), dtype=object)  # Größe angepasst, um Store i_Koordinate von 13 zu berücksichtigen

# Füllen Sie die Matrix mit den Ergebnissen Ihres Modells
for s in S:
    if x[s].solution_value() == 1:
        # Wenn ein Darkstore an diesem Standort errichtet wurde, markieren Sie die Zelle mit "DS"
        for q in Q:
            # Überprüfen Sie, ob der Quadrant von diesem Standort bedient wird

            if y[q, s].solution_value() == 1:
                # Wenn ja, fügen Sie die Nummer des beliefernden Darkstores hinzu
                ergebnis_matrix[q[0], q[1]] = str(s)
            elif y[q, s].solution_value() > 0:
                # Wenn der Quadrant bereits von einem anderen Darkstore beliefert wird
                if ergebnis_matrix[q[0], q[1]] is not None:
                    ergebnis_matrix[q[0], q[1]] += ',' + str(s)
                else:
                    ergebnis_matrix[q[0], q[1]] = str(s)


# Nachdem alle Quadranten bedient wurden, gehen Sie erneut über alle Standorte in S
for s in S:
    if x[s].solution_value() == 1:
        # Wenn ein Darkstore an diesem Standort errichtet wurde, markieren Sie die Zelle mit "DS"
        i, j = Koordinaten[s]['i_Koordinate'], Koordinaten[s]['j_Koordinate']
        if 0 <= i < 14 and 0 <= j < 14:  # Stellen Sie sicher, dass die Koordinaten innerhalb der Matrix liegen
            ergebnis_matrix[i, j] = "S" + str(s)

# Ausgabe der Ergebnismatrix
ergebnis_df = pd.DataFrame(ergebnis_matrix)
ergebnis_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,None,None,None,9,9,9,7,7,None,5,None,None,None,None
1,None,None,9,9,9,9,9,7,7,5,None,None,None,None
2,None,9,9,"7,9",9,7,7,9,7,7,None,5,None,None
3,9,9,9,9,9,7,9,9,7,7,7,None,None,None
4,9,7,9,9,S9,9,S7,9,S5,9,7,7,5,None
5,9,9,9,9,9,7,9,7,7,"5,7",7,5,5,None
6,None,9,9,9,7,9,9,9,7,7,1,5,None,None
7,None,None,9,9,9,9,5,7,7,5,5,3,3,None
8,None,None,None,9,9,9,7,"3,5,7",1,None,3,S1,None,None
9,None,None,None,None,9,2,4,None,2,None,3,3,None,None


####Visualisierung